## Benefits of encapsulation

- Protects data (data hiding) by preventing other parts of the program from accidentally changing important values. E.g. a user can’t directly set the rating to a negative number.

- Controls how data is changed, we can include validation or rules inside methods. e.g. only allow incrementRating() to increase by a sensible amount.
- Makes code easier to maintain, isolated changes to how the data is stored inside the class do not break other code.
- Improves readability and structure - it clarifies how data should be accessed — through getters and setters, not directly.
- Encourages reuse and modular design - with the class becoming self-contained and safely managing its own data.

### Encapsulation in the real world
Think of encapsulation like a hotel reception desk
- Guests (***other parts of the program***) cannot walk into the office and change room records directly
- They must ask the receptionist (***methods***) to update or view information properly.


In [10]:
class Customer:
    '''
- Customers are checked in an out of rooms
- customers leave feedback depending on how their stay was
- i.e. they are successfully checked in, room is clean
- Customers are happy if room is clearn
- Customers are unhappy if room is overbook or unclean'''
    def __init__(self, bookedRoom, name):
        '''class constructor'''
        self.__bookedRoom = bookedRoom
        self.__name = name
        self.__feedback =0
        self.__rating = 0 #added a rating
    
    '''Methods'''   
    def incrementRating (self, increment):
        '''setter for incrementing rating  / hotel experience'''
        self.__rating +=increment
        
    def getName(self):
        '''Returns the customer's name'''
        return self.__name
    def getRoom(self):
        '''Returns the booked room'''
        return self.__bookedRoom
    def getRating(self):
        '''Returns the rating'''
        return self.__rating

In [4]:
#Test the customer class

# Create a new customer
cust1 = Customer("Room 101", "Alice")

# Display initial details using the getters
print("Customer name:", cust1.getName())
print("Booked room:", cust1.getRoom())
print("Initial rating:", cust1.getRating())

# Simulate a good stay
cust1.incrementRating(4)
print("Updated rating after good stay:", cust1.getRating())

# Simulate another experience
cust1.incrementRating(1)
print("Final rating:", cust1.getRating())

Customer name: Alice
Booked room: Room 101
Initial rating: 0
Updated rating after good stay: 4
Final rating: 5


In [12]:
class Room:
    """
    - we may add occupants to a room
    - remove occupants
    - clean a room
    - get a room number
    """
    def __init__(self, number, size, clean):
        """Constructor method for the room object"""
        self.__number = number
        self.__size = size
        self.__occupants = [] # this is a list of objects i.e. people in the room
        self.__clean = clean

    def addOccupant(self, occupantIn):
        """
        - Add a customer to the room
        - if the room capacity / size had not been exceeded
        - update the occupant's experience
        - update the cleaniliness of the room
        """
        if len(self.__occupants) < self.__size:
            #room is bookable
            self.__occupants.append(occupantIn)
            occupantIn.incrementRating(1) # positive experience
        else:
            #room not bookable
            occupantIn.incrementRating(-1) # negative experience
            return
        if self.__clean:
            occupantIn.incrementRating(1) # room is clean
        else:
            occupantIn.incrementRating(-1)
        self.__clean = False
    
    def removeOccupant(self, occupantOut):
        """handles removal of an occupant, note that occupantOut is an object too"""
        index = -1  #stores the index of the occupant to be removed, -1 means they are not in the list
        for pos, occupant in enumerate(self.__occupants):
            if occupantOut.getName() == occupant.getName():
                index = pos
        if index != -1:
            del self.__occupants[index]
            
        """
        The enumerate function allows us to get the index and the object from the objects' list
        example usage:
        items = ['apple','banana', 'cherry']
        for index, value in enumerate(items):
            print(f'Index: {index}, Value: {value}')
        
        Outputs
        Index: 0 Value: Apple
        Index: 1 Value: banana
        Index: 2 Value: cherry
        
        """
    def cleanRoom(self):
        """clean the room if it is empty"""
        if self.__occupants == []:
            self.__clean = True
        return self.__clean
    def getNumber(self):
        """returns the room number - needed when booking the room"""
        return self.__number

### Room class follow on explanation
- inside removeOccupant() we want to remove a specific customer (***object***)  from the list of people currently in the room
- enumerate() goes through the list while keeping track of both:
  - pos: the index position in the list
  - occupant: the actual customer object
---
```python
  for pos, occupant in enumerate(self.__occupants):
```
- Returns
```python
  # if self.__occupants = [Alice, Bob, Chloe]
  pos = 0, occupant = Alice
  pos = 1, occupant = Bob
  pos = 2, occupant = Chloe
```
---
if occupantOut.getName() == occupant.getName():
  index = pos

- if the name of customer we are trying to remove is the same name as the one currently being checked, it means we found them  so we record their position (***pos***) by setting the index.


In [13]:
# Make a couple of customers
alice = Customer("Room 101", "Alice")
bob   = Customer("Room 101", "Bob")

# Make a room with capacity 1 and initially clean
room = Room(101, 1, True)

# Add Alice (should succeed, room becomes not clean)
room.addOccupant(alice)
print("Room number:", room.getNumber())
print("Alice rating:", alice.getRating())

# Try to add Bob (should fail and penalise Bob due to overcapacity)
room.addOccupant(bob)
print("Bob rating after attempt:", bob.getRating())

# Remove Alice, then clean the room
room.removeOccupant(alice)
print("Clean after emptying:", room.cleanRoom())

Room number: 101
Alice rating: 2
Bob rating after attempt: -1
Clean after emptying: True


In [15]:
class Hotel:
    """Hotel with methids """
    def __init__(self, rooms, customers, receptionist, cleaner, manager):
        """hotel constructor method"""
        self.__rooms = rooms
        self.__customers = customers
        self.__receptionist = receptionist
        self.__cleaner = cleaner
        self.__manager = manager

    def cleanRooms(self):
        """
        - method for cleaning the room
        - iterate over the rooms and clean the room if it needs to be cleaned
        """
        for room in self.__rooms:
            if room.cleanRoom():
                print(f'{self.__cleaner} cleaned Room {room.getNumber()}')
    def checkIn(self, customers):
        """used to check in a customer"""
        for customer in customers:
            room  = self.__rooms[self.__customers[customer-1].getRoom() -1]
            room.addOccupant(self.__customers[customer -1])
            print(f'{self.__receptionist} checked in {self.__customers[customer -1].getName()}')
    def checkOut(self, customers):
        #used to check in a customer
        for customer in customers:
            room  = self.__rooms[self.__customers[customer-1].getRoom() -1]
            room.removeOccupant(self.__customers[customer -1])
            print(f'{self.__receptionist} checked out {self.__customers[customer -1].getName()}')
            self.takeFeedback(self.__customers[customer -1])
    def takeFeedback(self, customer):
        #take customer feedback
        if customer.getRating() >0:
            print(f'{self.__manager} says:\n{customer.getName()} was happy with their stay!')
        elif customer.getRating() <0:
            print(f'{self.__manager} says:\n{customer.getName()} was unhappy with their stay!')
        else:
            print(f'{self.__manager} says:\n{customer.getName()} found their stay to be ok.')
    def checkRooms(self):
        return self.__rooms